# Setup


In [4]:
import os
from openai import OpenAI
import sys

sys.path.append("../..")
import utils
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())  # read local .env file

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [5]:
def get_completion_from_messages(
    messages, model="gpt-3.5-turbo", temperature=0, max_tokens=500
):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message.content

# OpenAI Function Calling


In [6]:
import json


# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [7]:
# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [8]:
messages = [{"role": "user", "content": "What's the weather like in Boston?"}]

In [9]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613", messages=messages, functions=functions
)

In [10]:
print(response)

ChatCompletion(id='chatcmpl-9UfJUNCiTb1Ge693inCiLNykTnXAN', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None))], created=1717095676, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=18, prompt_tokens=82, total_tokens=100))


In [12]:
response_message = response.choices[0].message

In [15]:
response_message

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None)

In [19]:
response_message.content

In [18]:
response_message.function_call

FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather')

In [21]:
json.loads(response_message.function_call.arguments)

{'location': 'Boston, MA'}

In [23]:
args = json.loads(response_message.function_call.arguments)

In [24]:
get_current_weather(args)

'{"location": {"location": "Boston, MA"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

In [25]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

In [26]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
)

In [27]:
print(response)

ChatCompletion(id='chatcmpl-9UfLfCXS5iUuFXo0uJw4T7Y69wJTd', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1717095811, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=10, prompt_tokens=76, total_tokens=86))


In [29]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",
)
print(response)

ChatCompletion(id='chatcmpl-9UfLu9tnbPNZ69GEH2fSL7g4ryg4y', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1717095826, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=10, prompt_tokens=76, total_tokens=86))


In [30]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response)

ChatCompletion(id='chatcmpl-9UfMEsgd5kp03p9K6iKT5CB4qVOAY', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1717095846, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=9, prompt_tokens=77, total_tokens=86))


In [31]:
messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response)

ChatCompletion(id='chatcmpl-9UfMi77AQIf9vcuGrugfVBR1umeT0', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Please give me a moment. I will fetch the current weather in Boston for you.', role='assistant', function_call=None, tool_calls=None))], created=1717095876, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=17, prompt_tokens=82, total_tokens=99))


In [32]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

ChatCompletion(id='chatcmpl-9UfN33A2lfodmDfoz8dIBYN7MGnty', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "San Francisco, CA"\n}', name='get_current_weather'), tool_calls=None))], created=1717095897, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=12, prompt_tokens=83, total_tokens=95))


In [33]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston!",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

ChatCompletion(id='chatcmpl-9UfNW5qZsp6Z3iyUvTLurZJT0y0rt', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None))], created=1717095926, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=11, prompt_tokens=89, total_tokens=100))


In [34]:
messages.append(response.choices[0].message)

In [35]:
args = json.loads(response.choices[0].message.function_call.arguments)
observation = get_current_weather(args)

In [36]:
messages.append(
    {
        "role": "function",
        "name": "get_current_weather",
        "content": observation,
    }
)

In [37]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
)
print(response)

ChatCompletion(id='chatcmpl-9UfOauWZO45z23GrlglmpI4GJs4Qa', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The current weather in Boston, MA is sunny and windy with a temperature of 72°F.', role='assistant', function_call=None, tool_calls=None))], created=1717095992, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=19, prompt_tokens=77, total_tokens=96))
